## Lesosn 21

[Course Repo](https://github.com/fastai/course22p2)


Breif outline

-  Jono: Demo of WandB.  (notebook 21_cifar10_and_wandb.ipynb) 
    - Uses CFAR10 instead of fashion mnist.  The tools still work. 
    - Weights and Biases (WandB) mointoring. Using [WandB](https://wandb.ai/site) for monitoring and tracking of model performance. 
    - You call logging functions in your code to log metrics, hyperparameters, and other data to the WandB server, and then you can view and compare these results in the WandB dashboard.
    - You can also use WandB to log models, code and more.
    - Jono implements a WandB callback in a few lines to implment this!
    - Jeremy says he doesn't use this (intentionally) because he worries about just doing wide sweeps of hyperparameters.  He prefers to do a more focused search.

- 20:50 Metrics  